# SELECTING BEST MODEL

In [31]:
import numpy as np
import pandas as pd

# for model selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# for cross validation
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
from collections import defaultdict

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [24]:
usecols = ['Customer_ID', 'Saving_account', 'Guarantees', 'Cur_account', 'Derivative_account', 'Payroll_account',
                'Junior_account', 'Particular_acct1', 'Particular_acct2', 'Particular_acct3', 'Short_term_deposites',
                'Med_term_deposites', 'Long_term_deposites', 'e-account', 'Funds', 'Mortgage', 'Pension', 'Loans',
                'Taxes', 'Credit_card', 'Securities', 'Home_account', 'Payroll', 'Pensions', 'Direct_debit']
       
df_train_1 = pd.read_csv("Train_Cleaned.csv",dtype={"Sex":str,
                                                    "New_customer":str,
                                                    "Last_date_Prim_Cust":str,
                                                    "Forigner_flag":str},usecols=usecols)

df_train_1 = df_train_1.drop_duplicates(['Customer_ID'], keep='last')
#df_test = df_test.drop_duplicates(['ncodpers'], keep='last')
df_train_1.fillna(0, inplace=True)
#df_test.fillna(0, inplace=True)

#We take unique_ids and unique fecha_datos just to play forward.
unique_ids   = pd.Series(df_train_1["Customer_ID"].unique())

df_train_1.count()
unique_ids.count()
limit_people = 10000
unique_id    = unique_ids.sample(n=limit_people)
df_train           = df_train_1[df_train_1.Customer_ID.isin(unique_id)]

In [25]:
models = {}
id_preds = defaultdict(list)
ids = df_train['Customer_ID'].values
   
for c in df_train.columns:
    if c != 'Customer_ID':
        #print(c)
        y_train = df_train[c]
        x_train = df_train.drop([c, 'Customer_ID'], 1)
        
        
for c in df_test_1.columns:
    if c != 'Customer_ID':
        #print(c)
        x_test = df_test_1.drop([c, 'Customer_ID'], 1) 
        x_test = x_test.fillna(method='ffill')

In [40]:
x_train.head(10)

,Saving_account,Guarantees,Cur_account,Derivative_account,Payroll_account,Junior_account,Particular_acct1,Particular_acct2,Particular_acct3,Short_term_deposites,Med_term_deposites,Long_term_deposites,e-account,Funds,Mortgage,Pension,Loans,Taxes,Credit_card,Securities,Home_account,Payroll,Pensions
15744,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158914,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
164543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
192560,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
199488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
206795,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
245281,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
336547,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
552075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
models = [('lr',LogisticRegression()),('gbr',GradientBoostingClassifier()),
          ('xgb',XGBClassifier()),('knn',KNeighborsClassifier())]

In [38]:
#Making function for making best 2 models for further hyperparameter tuning
def basic_model_selection(x,y,model,z):
    scores=[]
    names = []
    for i , j in model:
        clf = j
        clf.fit(x,y)
        p_train = clf.predict_proba(x)[:,1]
        for id,p in zip(ids,p_train):
            id_preds[id].append(p)
        names.append(i)
        scores.append(roc_auc_score(y,p_train))
    for k in range(len(scores)):
        print(names[k],scores[k])

In [39]:
basic_model_selection(x_train,y_train,models,ids)

lr 0.865991925540692
gbr 0.8690092332508274
xgb 0.8683741691464805
knn 0.812363146721547
